In [6]:
import requests
import pickle
import pandas as pd
import time
import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

- 구현 필요 기능
1. API 연결 및 테스트
2. 요청 응답처리
3. API 연결 초기화
4. 통계 코드 검색기능

- 기능으로 구분
- 코드 검색 함수
- Client
- test?

In [ ]:
class stats_codes:
    def __init__(self):
        self.stats_codes_info = None
        self.stats_codes = None
    
    def load_stats_code(self, path):
        with open(path,"rb") as f:
            self.stats_codes_info = pickle.load(f)
        self.stats_codes_info = pd.DataFrame(self.stats_codes_info)

    def update_stats_code(self, api_key):
        self.crawling_stats_code()
        stats_list = []
        idx_no = 0
        for stats_code in tqdm(self.stats_codes, desc="Processing stats codes"):
            # url 지정
            url = 'https://ecos.bok.or.kr/api/StatisticItemList/{}/json/kr/1/10/{}'.format(api_key,stats_code)
            
            # API 호출
            response = requests.get(url)
            
            # 응답 데이터 확인
            if response.status_code == 200:
                data = response.json()  # JSON 데이터를 Python 딕셔너리로 변환
            else:
                print(f"Error: {response.status_code}, {response.text}")
            
            idx_no += 1
            if idx_no % 100 == 0:
                print(idx_no, len(stats_list))
                time.sleep(60)

            try : stats_list = stats_list + data['StatisticItemList']['row']
            except : pass

        self.stats_codes_info = pd.DataFrame(stats_list)


    def crawling_stats_code(self):
        # 크롬 드라이버 설정
        chrome_options = Options()
        #chrome_options.add_argument("--headless")  # 브라우저 창을 열지 않고 실행
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # 드라이버 초기화
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            # 웹 페이지 열기
            url = "https://ecos.bok.or.kr/api/#/DevGuide/StatisticalCodeSearch"  # 크롤링할 URL
            driver.get(url)
            # 스크롤 후 대기
            time.sleep(3)
            # 스크롤 대상 div.rg-scrolltrack 찾기
            scroll_down = driver.find_element(By.CLASS_NAME, "rg-scroll-down")
            # 이미 찾은 mark 태그를 저장할 집합
            found_marks = set()
            # 스크롤하면서 mark 태그 찾기
            while True:
                pre_marks_num = len(found_marks) 
                for i in range(10):
                    scroll_down.click()
                # 스크롤 후 대기
                time.sleep(0.2)
                # mark 태그 찾기
                marks = driver.find_elements(By.CSS_SELECTOR, "mark")
                for mark in marks:
                    found_marks.add(mark.text)
                # 스크롤이 더 이상 움직이지 않을 경우 종료
                if pre_marks_num == len(found_marks) :
                    break
            
            self.stats_codes = [i.split('[')[1].split(']')[0] for i in list(found_marks)]

        finally:
            pass

        print('통계 코드 총 {} 개 확인'.format(len(self.found_marks)))





